<a href="https://colab.research.google.com/github/Goodbam/Changing-Naver-rising-searches/blob/master/crawling(everyone).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install lxml
!pip install parse

    100% |████████████████████████████████| 5.8MB 6.5MB/s 
  Running setup.py bdist_wheel for parse ... - done
  Stored in directory: /root/.cache/pip/wheels/ef/db/c6/18568a2cc574848f3996ac4552241fbec046b7be29feb2077d
Successfully built parse


In [0]:
import requests                 # url request
from lxml.html import parse     # parsing the data as the html form
from io import StringIO         # String I/O module

THRESHOLD = 25 #limit to check whether they are relative or not

In [3]:
url = "https://www.naver.com"

# Get html source
text = requests.get(url).text

# parsing the data as the html form
text_source = StringIO(text)    # read the data as string
parsed = parse(text_source)     # source ->parsing the data as the html form 

#find root node
doc = parsed.getroot()

# doc.findall(".//TagName") => find the tag lists
uls = doc.findall('.//ul')

#Get the Rising searches
ul = uls[10]   
spans = ul.findall('.//span')

data_list= []  # to save all the keywords

cnt = 1
for data in spans:
  if cnt % 2 == 0:  
    print(int((cnt+1)/2),'->',data.text_content().strip())  # show the existing rising searches
  cnt +=1  
  data_list.append(data.text_content().strip()) #save each keyword
    


1 -> sbs 드라마
2 -> 나쁜형사
3 -> 이커머스
4 -> 2019 수능 등급컷
5 -> 이선정
6 -> 이설
7 -> 수능 등급컷
8 -> 홍영표
9 -> 배지현
10 -> 캡틴 마블
11 -> 붐붐파워
12 -> 진학사
13 -> 수능 성적 발표
14 -> 노니
15 -> 이일재
16 -> 황지만
17 -> 노니 쇳가루
18 -> 정재형
19 -> 식품안전나라
20 -> 광주형 일자리


In [4]:
path='https://search.naver.com/search.naver?sm=top_hty&fbm=1&ie=utf8&query=' #This is the url for surfing

matrix = [] #to count the score of relationship
label = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] #disjoint set

cnt1 = 0

for i in range(0, 20):
  word = data_list[1 + cnt1]
  search = path + word #url searching the keyword
  text = requests.get(search).text # no worry about encoding url
  
  row = []
  cnt2 = 0
  for j in range(0, 20):
    if cnt1 != cnt2: #no need to check when they are equivalent
      target = data_list[1 + cnt2];
      row.append(text.count(target) + 7*word.count(target)) 
 
      if row[int(cnt2/2)] >= THRESHOLD: #IF they are relative,
        print(word, ' and ', target,)
        #the representive number should be minimized
        
        #여기 정규화를 해서 row의합으로 각 row의 값을 나누던지 해보자
        #그러려니, 전체적으로 점수 자체가 높은 것은 감안해서 평균을 다시 루트씌우고 곱하든지 고려해보자
        #A와 B 관련 , B C 관련 이면 A C는 관련이 있는건가? 이것도 감안
        
        if label[int(cnt1/2)] < label[int(cnt2/2)]:
          label[int(cnt2/2)] = label[int(cnt1/2)]  
        
        else:
          label[int(cnt1/2)] = label[int(cnt2/2)]
    
    else :
      row.append(0)
    
    cnt2 += 2
  
  matrix.append(row)  
  cnt1 += 2
  

나쁜형사  and  이설
2019 수능 등급컷  and  수능 등급컷
이설  and  나쁜형사
수능 등급컷  and  2019 수능 등급컷
진학사  and  2019 수능 등급컷
진학사  and  수능 등급컷
수능 성적 발표  and  수능 등급컷
황지만  and  이선정
노니 쇳가루  and  노니
식품안전나라  and  노니


In [5]:
print('\n',label,'\n')

print(matrix)

score =[0]
score = score*20;

for i in range(0,20):
  score[label[i] - 1] += 1/(i+1); #label is 1~20 => needed to subtract 1


 [1, 2, 3, 4, 5, 2, 4, 8, 9, 10, 11, 4, 4, 14, 15, 5, 14, 18, 14, 20] 

[[0, 11, 9, 11, 9, 9, 20, 9, 9, 9, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 0, 9, 11, 9, 26, 20, 9, 9, 9, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 0, 11, 9, 9, 20, 9, 9, 9, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 0, 9, 9, 115, 9, 9, 9, 9, 23, 14, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 11, 0, 9, 20, 9, 9, 9, 9, 9, 11, 20, 11, 10, 11, 9, 9, 9], [11, 108, 9, 11, 9, 0, 20, 9, 9, 9, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 35, 9, 9, 0, 9, 9, 9, 9, 14, 15, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 11, 9, 9, 20, 0, 9, 9, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 11, 9, 9, 20, 9, 0, 9, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 11, 9, 9, 20, 9, 9, 0, 9, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 11, 9, 9, 20, 9, 9, 9, 0, 9, 11, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 31, 9, 9, 43, 9, 9, 9, 9, 0, 14, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 19, 9, 9, 33, 9, 9, 9, 9, 17, 0, 20, 11, 9, 11, 9, 9, 9], [11, 9, 9, 11, 9, 9, 20,

In [6]:
#RESULT
print('\n☆급상승 검색어 순위☆\n')
cnt = 1;

#sorting according to SCORE
for i in range(0,20):
  max = 0
  for j in range(1,20):
    if score[j] > score[max]:
      max = j;
  
  relative_word = ''
  
  for j in range(0,20):
    if label[j] == max + 1:
      relative_word += data_list[1 + 2*j] + ' | ' 
  
  score[max] = 0;
  
  print(cnt, '. ', relative_word[0:-3])
  cnt += 1
  if(score.count(0) == 20):
    break  


☆급상승 검색어 순위☆

1 .  sbs 드라마
2 .  나쁜형사 | 이설
3 .  2019 수능 등급컷 | 수능 등급컷 | 진학사 | 수능 성적 발표
4 .  이커머스
5 .  이선정 | 황지만
6 .  노니 | 노니 쇳가루 | 식품안전나라
7 .  홍영표
8 .  배지현
9 .  캡틴 마블
10 .  붐붐파워
11 .  이일재
12 .  정재형
13 .  광주형 일자리
